In [1]:
import pandas as pd
import math
import statistics
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

In [2]:
def split_dataframe_by_percentage(df, percentage):
    if not 0 <= percentage <= 1:
        raise ValueError("Percentage should be between 0 and 1")
    num_rows_first_part = int(len(df) * percentage)
    first_part = df.iloc[:num_rows_first_part]
    second_part = df.iloc[num_rows_first_part:]
    
    return first_part, second_part

In [3]:
df_wm = pd.read_csv('./features_19/features_19_WM.csv', parse_dates=['date'])
df_wm=df_wm.T.drop("date").T.astype(float)

C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\964684434.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_wm = pd.read_csv('./features_19/features_19_WM.csv', parse_dates=['date'])


In [4]:
df_ov = pd.read_csv('./features_19/features_19_OVEN.csv', parse_dates=['date'])
df_ov= df_ov.T.drop("date").T.astype(float)

C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\2267688325.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ov = pd.read_csv('./features_19/features_19_OVEN.csv', parse_dates=['date'])


In [5]:
df_tb  = pd.read_csv('./features_19/features_19_TB.csv', parse_dates=['date'])
df_tb= df_tb.T.drop("date").T.astype(float)

C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\2711163810.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tb  = pd.read_csv('./features_19/features_19_TB.csv', parse_dates=['date'])


In [6]:
df_tb= df_tb.T.drop(["zeros"]).T
df_ov= df_ov.T.drop(["zeros"]).T
df_wm =df_wm.T.drop(["zeros"]).T

In [7]:
binary_clasif = False

In [8]:
x=300
df_tb =df_tb.head(x)
df_ov =df_ov.head(x)
df_wm =df_wm.head(x)

In [9]:
#0.3
trainT_x,testT_x = split_dataframe_by_percentage(df_tb, 0.5)
trainO_x,testO_x = split_dataframe_by_percentage(df_ov, 0.5)
trainW_x,testW_x = split_dataframe_by_percentage(df_wm, 0.5)

In [10]:
len(trainW_x)

127

In [11]:
len(trainO_x)

134

In [12]:
nr_real_ts =len(df_tb)+len(df_ov)+len(df_wm)
nr_real_ts

780

In [13]:
if binary_clasif: 
    train_x= pd.concat([trainO_x, trainW_x], axis=0)  
    test_x = pd.concat([testO_x, testW_x], axis=0)  
else:
    train_x= pd.concat([trainT_x, trainO_x, trainW_x], axis=0)  
    test_x = pd.concat([testT_x, testO_x, testW_x], axis=0)  
train_x

,mean,std,25%,50%,75%,max,Mon,Tue,Wed,Thu,Fri,Sat,Sun,trend_angle,shannon,skewness,kurtosis,min_poz
0,19894.619178,24459.626561,0.0,16325.0,27092.0,125251.0,27542.867925,21787.076923,17050.346154,19410.750000,21374.250000,12506.173077,19443.788462,-74.611128,5.445766,1.494442,2.234470,400.0
1,25640.912329,32608.253221,0.0,8687.0,46943.0,147733.0,21511.735849,17080.692308,33034.788462,38569.673077,28971.134615,22716.711538,17681.057692,-89.395749,5.469615,1.124894,0.196796,334.0
2,17919.569863,23392.812468,0.0,9967.0,28696.0,122466.0,24703.509434,18571.826923,35298.673077,11726.557692,9251.423077,11320.846154,14433.692308,82.980477,5.440287,1.537214,2.011798,588.0
3,35469.638356,34188.125118,0.0,31178.0,56832.0,118514.0,20953.358491,39279.634615,46406.057692,34575.423077,32962.019231,40078.153846,34311.980769,89.058454,6.752420,0.603655,-0.855129,642.0
4,14034.284932,13619.316965,0.0,14166.0,20749.0,90136.0,13893.886792,14363.384615,15568.019231,13665.480769,15374.673077,10975.500000,14401.750000,-81.858799,6.143752,1.225726,2.802443,5746.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,5885.610959,5731.757237,0.0,5828.0,9446.0,24624.0,5329.377358,5894.038462,5189.192308,6162.769231,5240.096154,6276.307692,7118.192308,30.653405,6.207143,0.704032,-0.313693,1167.0
123,3509.800000,4024.863282,0.0,3558.0,4754.0,20469.0,2555.867925,3292.576923,6637.153846,3341.769231,3450.403846,2187.730769,3121.442308,-74.706059,5.436575,1.406717,2.227875,861.0
124,8848.761644,10037.905519,0.0,7244.0,14946.0,43983.0,9418.037736,7186.403846,9444.557692,8542.884615,10244.211538,7667.307692,9426.980769,-85.154866,5.601200,1.140001,0.781597,1211.0
125,4488.526027,5162.726431,0.0,3500.0,7067.0,24400.0,4966.433962,4586.961538,8054.096154,3232.711538,3956.346154,3962.326923,2651.615385,-72.643473,5.611488,1.291261,1.335118,3322.0


In [14]:
trainT_y = np.zeros(len(trainT_x.T.columns))  # all columns in df_td belong to tumbler dryer 0
trainO_y = np.ones(len(trainO_x.T.columns))   # all columns in df_ov belong to oven 1
trainW_y = 2 * np.ones(len(trainW_x.T.columns))  # all columns in df_wm belong to washing machine 2

testT_y = np.zeros(len(testT_x.T.columns))  # all columns in df_td belong to tumbler dryer 0
testO_y = np.ones(len(testO_x.T.columns))   # all columns in df_ov belong to oven 1
testW_y = 2 * np.ones(len(testW_x.T.columns))  # all columns in df_wm belong to washing machine 2

if binary_clasif: 
    train_y= np.concatenate([trainO_y, trainW_y], axis=0)  
    test_y = np.concatenate([testO_y, testW_y], axis=0)  
else:
    train_y= np.concatenate([trainT_y, trainO_y, trainW_y], axis=0)  
    test_y = np.concatenate([testT_y, testO_y, testW_y], axis=0)  
    
    
print("Dimensiune train:",len(train_x), " cu dimensiine label:" ,len(train_y))
print("Dimensiune test:",len(test_x), " cu dimensiine label:" ,len(test_y))

Dimensiune train: 389  cu dimensiine label: 389
Dimensiune test: 391  cu dimensiine label: 391


## add agument

In [15]:
df_wm_s = pd.read_csv('./features_19/features_19_WM_GEN.csv', parse_dates=['date'])
df_wm_s=df_wm_s.T.drop("date").T.astype(float)


df_ov_s = pd.read_csv('./features_19/features_19_OVEN_GEN.csv', parse_dates=['date'])
df_ov_s = df_ov_s.T.drop("date").T.astype(float)

df_tb_s = pd.read_csv('./features_19/features_19_TB_GEN.csv', parse_dates=['date'])
df_tb_s = df_tb_s.T.drop("date").T.astype(float)


C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\4218104544.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_wm_s = pd.read_csv('./features_19/features_19_WM_GEN.csv', parse_dates=['date'])
C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\4218104544.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ov_s = pd.read_csv('./features_19/features_19_OVEN_GEN.csv', parse_dates=['date'])
C:\Users\Iulia\AppData\Local\Temp\ipykernel_25644\4218104544.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tb_s = pd.read_csv('./features_19/features_19_TB_GEN.csv', parse_dat

In [16]:
#["zeros","max/min","mean/std"]
df_tb_s= df_tb_s.T.drop(["zeros"]).T
df_ov_s= df_ov_s.T.drop(["zeros"]).T
df_wm_s =df_wm_s.T.drop(["zeros"]).T

In [17]:
len(trainO_x)

134

In [18]:
len(trainW_x)

127

In [19]:
df_tb_s = df_tb_s.head(int(len(trainT_x)))
df_ov_s = df_ov_s.head(int(len(trainO_x)))
df_wm_s = df_wm_s.head(int(len(trainW_x)))

In [20]:
trainW_x

,mean,std,25%,50%,75%,max,Mon,Tue,Wed,Thu,Fri,Sat,Sun,trend_angle,shannon,skewness,kurtosis,min_poz
0,5942.893151,7410.906028,0.0,1858.0,10443.0,38042.0,9895.188679,6158.673077,4367.846154,5220.884615,4999.134615,5253.865385,5628.653846,-59.655071,5.736809,1.307190,1.170905,877.0
1,6427.106849,4801.336366,3693.0,5456.0,9234.0,30243.0,6914.132075,7176.788462,6586.807692,6984.269231,6342.884615,5149.846154,5825.653846,7.229326,7.276142,0.727575,1.184914,871.0
2,6540.282192,6075.742744,0.0,7561.0,9774.0,25722.0,7470.773585,7902.288462,7734.057692,5566.557692,6972.903846,5382.961538,4734.538462,-85.701960,5.766274,0.512504,-0.405627,5168.0
3,7734.421918,7326.187076,0.0,6060.0,13997.0,34011.0,8714.754717,6892.038462,8987.846154,8354.038462,9098.788462,4502.788462,7571.846154,-86.400838,6.241935,0.763757,-0.032864,4135.0
4,9239.695890,7180.872033,4360.0,7658.0,13482.0,31070.0,9206.396226,9562.788462,9132.480769,9486.615385,8863.750000,9079.865385,9346.615385,72.618500,7.230932,0.554961,-0.106035,499.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,5885.610959,5731.757237,0.0,5828.0,9446.0,24624.0,5329.377358,5894.038462,5189.192308,6162.769231,5240.096154,6276.307692,7118.192308,30.653405,6.207143,0.704032,-0.313693,1167.0
123,3509.800000,4024.863282,0.0,3558.0,4754.0,20469.0,2555.867925,3292.576923,6637.153846,3341.769231,3450.403846,2187.730769,3121.442308,-74.706059,5.436575,1.406717,2.227875,861.0
124,8848.761644,10037.905519,0.0,7244.0,14946.0,43983.0,9418.037736,7186.403846,9444.557692,8542.884615,10244.211538,7667.307692,9426.980769,-85.154866,5.601200,1.140001,0.781597,1211.0
125,4488.526027,5162.726431,0.0,3500.0,7067.0,24400.0,4966.433962,4586.961538,8054.096154,3232.711538,3956.346154,3962.326923,2651.615385,-72.643473,5.611488,1.291261,1.335118,3322.0


In [21]:
df_wm_s

,mean,std,25%,50%,75%,max,Mon,Tue,Wed,Thu,Fri,Sat,Sun,trend_angle,shannon,skewness,kurtosis,min_poz
0,6147.081389,6320.178618,0.000000,5718.155380,9896.374663,26610.0,10050.000525,7259.966972,4792.048181,4982.007217,4703.312989,5741.680178,5425.497526,-36.393189,5.777233,0.863376,0.173551,352.900771
1,6373.812586,4269.621640,3747.216137,6310.506102,9226.082224,17726.0,6623.916203,7273.674069,6215.206835,6469.485590,6393.099922,5455.257335,6181.238461,-21.527781,7.356990,0.106847,-0.589223,238.326933
2,5590.506213,5300.504291,0.000000,6048.625180,9151.060002,19490.0,6267.255355,7307.470777,6898.110940,4373.215498,5816.534819,4129.956338,4327.985359,-84.618604,5.745426,0.473956,-0.728127,259.859360
3,7278.741183,6406.357348,0.000000,7308.606123,11541.032540,24991.0,8825.449309,5072.802468,8041.123727,8168.962573,9109.884899,4577.889926,7125.330994,-86.377736,6.250826,0.454806,-0.592701,1718.331779
4,9014.449054,6594.555692,4525.858105,8881.479889,13261.727040,26710.0,9090.560774,8817.458132,9133.242054,8942.083907,9605.918235,8524.367376,8986.049213,58.710614,7.230932,0.390981,-0.346500,2537.343750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,5823.345726,5122.139248,0.000000,5784.512385,9602.714282,19306.0,5051.015630,4983.379421,5550.899905,6482.491712,5087.868664,6391.571323,7231.045930,78.702797,6.196184,0.396562,-0.750417,762.874928
123,3330.003681,3398.094665,0.000000,3515.857360,5757.958586,12336.0,2119.400969,3710.480316,5506.541322,3174.483165,3610.611973,2417.305634,2794.483206,-74.817095,5.461630,0.615568,-0.573040,33.371037
124,8022.289545,8139.415599,0.000000,7703.877332,13717.869183,32724.0,8499.195159,7190.880163,9246.251908,7382.085187,7847.602662,7959.068753,8021.771722,-82.996288,5.606679,0.664450,-0.365420,426.140681
125,3970.626450,4191.033102,0.000000,3473.677863,6390.396791,18297.0,3961.638348,3561.529183,6375.301065,3919.434622,3928.482005,3556.841602,2491.331172,-67.078339,5.659460,0.930014,0.276344,1238.557592


In [22]:
if binary_clasif: 
    train_synt_x= pd.concat([df_ov_s, df_wm_s], axis=0)  
else:
    train_synt_x= pd.concat([df_tb_s, df_ov_s, df_wm_s], axis=0)  
train_synt_x

,mean,std,25%,50%,75%,max,Mon,Tue,Wed,Thu,Fri,Sat,Sun,trend_angle,shannon,skewness,kurtosis,min_poz
0,19015.160668,20569.629896,0.0,18248.225907,31569.006948,74345.000000,26725.265728,23853.355488,14080.463578,15516.478047,22249.961539,13453.234684,17079.094361,71.785356,5.413959,0.850121,-0.123866,376.229899
1,21032.258929,23531.521074,0.0,14843.233794,39660.501088,102936.981500,16526.845667,15977.081126,25568.376713,33537.936214,23200.555809,18007.746666,14493.912872,-89.402616,5.469615,0.858287,-0.330596,598.140154
2,18937.831585,20736.259436,0.0,16329.561606,31723.205218,87587.000000,25685.669338,20023.229629,34726.848059,11723.299993,12908.096112,12101.048213,15266.863639,86.662820,5.440287,0.948648,0.301518,1590.701076
3,36861.666603,30270.506958,0.0,36130.108635,59053.721444,106217.000000,26638.783756,37216.122174,52229.634445,34574.529389,33913.569401,39682.374961,33973.245997,89.063183,6.714064,0.338139,-0.842779,1329.353147
4,13915.716853,12648.587432,0.0,13366.170298,22737.359538,48774.473217,12816.578902,13147.653470,18123.888158,13343.054922,14939.120391,9283.036061,15777.823336,-82.069334,6.149231,0.512242,-0.523734,401.047578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,5823.345726,5122.139248,0.0,5784.512385,9602.714282,19306.000000,5051.015630,4983.379421,5550.899905,6482.491712,5087.868664,6391.571323,7231.045930,78.702797,6.196184,0.396562,-0.750417,762.874928
123,3330.003681,3398.094665,0.0,3515.857360,5757.958586,12336.000000,2119.400969,3710.480316,5506.541322,3174.483165,3610.611973,2417.305634,2794.483206,-74.817095,5.461630,0.615568,-0.573040,33.371037
124,8022.289545,8139.415599,0.0,7703.877332,13717.869183,32724.000000,8499.195159,7190.880163,9246.251908,7382.085187,7847.602662,7959.068753,8021.771722,-82.996288,5.606679,0.664450,-0.365420,426.140681
125,3970.626450,4191.033102,0.0,3473.677863,6390.396791,18297.000000,3961.638348,3561.529183,6375.301065,3919.434622,3928.482005,3556.841602,2491.331172,-67.078339,5.659460,0.930014,0.276344,1238.557592


In [23]:
labels_td = np.zeros(len(df_tb_s.T.columns))  # all columns in df_td belong to tumbler dryer 0
labels_ov = np.ones(len(df_ov_s.T.columns))   # all columns in df_ov belong to oven 1
labels_wm = 2 * np.ones(len(df_wm_s.T.columns))  # all columns in df_wm belong to washing machine 2

if binary_clasif: 
    train_synt_y = np.concatenate((labels_ov, labels_wm)) 
else:
    train_synt_y = np.concatenate((labels_td,labels_ov, labels_wm))

print("Dimensiune target:",len(train_synt_y))
print("Dimensiune date: ",len(train_synt_x) )

Dimensiune target: 389
Dimensiune date:  389


In [24]:
trainAgg_x = pd.concat([train_x, train_synt_x], axis=0)  
trainAgg_y = np.concatenate((train_y, train_synt_y)) 
print(len(trainAgg_x), len(trainAgg_y))

778 778


In [25]:
len(train_x)

389

In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x.values.reshape(train_x.shape[0], -1)).reshape(train_x.shape[0], train_x.shape[1], 1)
trainAgg_x = scaler.transform(trainAgg_x.values.reshape(trainAgg_x.shape[0], -1)).reshape(trainAgg_x.shape[0], trainAgg_x.shape[1], 1)
test_x = scaler.transform(test_x.values.reshape(test_x.shape[0], -1)).reshape(test_x.shape[0], test_x.shape[1], 1)


In [27]:
def shuffle_data(train_x, train_y):
    permutation = np.random.permutation(len(train_x))
    shuffled_train_x = train_x[permutation]
    shuffled_train_y = train_y[permutation]
    
    return shuffled_train_x, shuffled_train_y


train_x, train_y = shuffle_data(train_x, train_y)
trainAgg_x, trainAgg_y = shuffle_data(trainAgg_x, trainAgg_y)
test_x, test_y = shuffle_data(test_x, test_y)

In [28]:
fold = 10

# ------RandomForestClassifier-----------------------------------------------------------------

In [242]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

acc_or = 0
acc_agg = 0
for x in range (10):
    
    train_x, train_y = shuffle_data(train_x, train_y)
    trainAgg_x, trainAgg_y = shuffle_data(trainAgg_x, trainAgg_y)
    test_x, test_y = shuffle_data(test_x, test_y)


    train_x_2d = train_x.reshape(train_x.shape[0], -1)
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(rf_classifier, train_x_2d, train_y, cv=fold)  # Assuming 5-fold cross-validation

    #print("Cross-validation scores:", scores)
    #print("Average accuracy:", scores.mean())
    if(scores.mean() > acc_or):
        acc_or = scores.mean()
    # Reshape the data
    train_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
    test_x_2d = test_x.reshape(test_x.shape[0], -1)
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(rf_classifier, train_x_2d, trainAgg_y, cv=fold)  

    #print("Cross-validation scores:", scores)
    #print("Average accuracy:", scores.mean())
    if(scores.mean() > acc_agg):
        acc_agg = scores.mean()
print("Cel mai buna accuratete pentru setul original: ",acc_or)
print("Cel mai buna accuratete pentru setul aggumentat: ",acc_agg)

Cel mai buna accuratete pentru setul original:  0.925506072874494
Cel mai buna accuratete pentru setul aggumentat:  0.898831092928112


In [103]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

train_x_2d = train_x.reshape(train_x.shape[0], -1)
test_x_2d = test_x.reshape(test_x.shape[0], -1)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_x_2d, train_y)
predictions = rf_classifier.predict(test_x_2d)
accuracy = accuracy_score(test_y, predictions)
print(f'Test Accuracy: {accuracy}')

conf_matrix = confusion_matrix(test_y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

train_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
test_x_2d = test_x.reshape(test_x.shape[0], -1)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_x_2d, trainAgg_y)
predictions = rf_classifier.predict(test_x_2d)

accuracy = accuracy_score(test_y, predictions)
print(f'Test Accuracy: {accuracy}')

conf_matrix = confusion_matrix(test_y, predictions)
print("Confusion Matrix:")
print(conf_matrix)


Test Accuracy: 0.9
Confusion Matrix:
[[35  4  1]
 [ 4 36  0]
 [ 3  0 37]]
Test Accuracy: 0.8916666666666667
Confusion Matrix:
[[35  5  0]
 [ 5 35  0]
 [ 3  0 37]]


## ---------NB---------------------------------------------------------------------------------------------------------------

In [243]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

acc_or = 0
acc_agg = 0
for x in range(10):
    train_x, train_y = shuffle_data(train_x, train_y)
    trainAgg_x, trainAgg_y = shuffle_data(trainAgg_x, trainAgg_y)
    test_x, test_y = shuffle_data(test_x, test_y)

    # Train and evaluate Gaussian Naive Bayes for original dataset
    nb_classifier = GaussianNB()
    train_x_2d = train_x.reshape(train_x.shape[0], -1)
    scores = cross_val_score(nb_classifier, train_x_2d, train_y, cv=fold)

    if scores.mean() > acc_or:
        acc_or = scores.mean()

    # Train and evaluate Gaussian Naive Bayes for augmented dataset
    nb_classifier = GaussianNB()
    trainAgg_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
    scores = cross_val_score(nb_classifier, trainAgg_x_2d, trainAgg_y, cv=fold)

    if scores.mean() > acc_agg:
        acc_agg = scores.mean()

print("Cel mai buna accuratete pentru setul original: ", acc_or)
print("Cel mai buna accuratete pentru setul augmentat: ", acc_agg)


Cel mai buna accuratete pentru setul original:  0.7790823211875842
Cel mai buna accuratete pentru setul augmentat:  0.7599941554646406


In [144]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


train_x_2d = train_x.reshape(train_x.shape[0], -1)
nb_classifier = GaussianNB()
scores = cross_val_score(nb_classifier, train_x_2d, train_y, cv=fold) 
print("Cross-validation scores:", scores)
print("Average accuracy:", scores.mean())

train_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
test_x_2d = test_x.reshape(test_x.shape[0], -1)
nb_classifier = GaussianNB()

scores = cross_val_score(nb_classifier, train_x_2d, trainAgg_y, cv=fold) 
print("Cross-validation scores:", scores)
print("Average accuracy:", scores.mean())


Cross-validation scores: [0.79487179 0.76923077 0.74358974 0.76923077 0.69230769 0.82051282
 0.82051282 0.74358974 0.84615385 0.78947368]
Average accuracy: 0.7789473684210526
Cross-validation scores: [0.80769231 0.67948718 0.78205128 0.73076923 0.64102564 0.82051282
 0.79487179 0.74358974 0.76623377 0.72727273]
Average accuracy: 0.7493506493506493


## ------SVM------------------------------------------------------------------------------------------------------------------

In [422]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

acc_or = 0
acc_agg = 0
kernels = ['linear', 'poly', 'rbf']
for k in  kernels:
    print("kernel", k)
    max_iter = 1000
    folds = 3
    for x in range(10):
        train_x, train_y = shuffle_data(train_x, train_y)
        trainAgg_x, trainAgg_y = shuffle_data(trainAgg_x, trainAgg_y)
        test_x, test_y = shuffle_data(test_x, test_y)

        # Train and evaluate SVM for original dataset
        svm_classifier = SVC(kernel=k,degree=7,max_iter=max_iter)
        train_x_2d = train_x.reshape(train_x.shape[0], -1)
        scores = cross_val_score(svm_classifier, train_x_2d, train_y, cv=fold)

        if scores.mean() > acc_or:
            acc_or = scores.mean()

        # Train and evaluate SVM for augmented dataset
        svm_classifier = SVC(kernel=k,degree=7,max_iter=max_iter)
        trainAgg_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
        scores = cross_val_score(svm_classifier, trainAgg_x_2d, trainAgg_y, cv=fold)

        if scores.mean() > acc_agg:
            acc_agg = scores.mean()

    print("Cel mai buna accuratete pentru setul original: ", acc_or)
    print("Cel mai buna accuratete pentru setul augmentat: ", acc_agg)


kernel linear
Cel mai buna accuratete pentru setul original:  0.8166666666666668
Cel mai buna accuratete pentru setul augmentat:  0.8222222222222223
kernel poly


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1

Cel mai buna accuratete pentru setul original:  0.8375
Cel mai buna accuratete pentru setul augmentat:  0.8222222222222223
kernel rbf
Cel mai buna accuratete pentru setul original:  0.8375
Cel mai buna accuratete pentru setul augmentat:  0.8305555555555557


In [386]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
kernels = ['linear', 'poly', 'rbf']
for k in  kernels:
    print("kernel", k)
    max_iter = 1000
    fold = 5
    degree = 2
    train_x_2d = train_x.reshape(train_x.shape[0], -1)
    svm_classifier = SVC(kernel=k,degree=degree,max_iter=max_iter)
    scores = cross_val_score(svm_classifier, train_x_2d, train_y, cv=fold)  

    print("Cross-validation scores:", scores)
    print("Average accuracy:", scores.mean())


    train_x_2d = trainAgg_x.reshape(trainAgg_x.shape[0], -1)
    test_x_2d = test_x.reshape(test_x.shape[0], -1)
    svm_classifier = SVC(kernel=k,degree=degree,max_iter=max_iter)

    scores = cross_val_score(svm_classifier, train_x_2d, trainAgg_y, cv=fold)  


    print("Cross-validation scores:", scores)
    print("Average accuracy:", scores.mean(),"\n")


kernel linear
Cross-validation scores: [0.85714286 0.78571429 0.71428571 0.78571429 0.88095238]
Average accuracy: 0.8047619047619048
Cross-validation scores: [0.79365079 0.73015873 0.82539683 0.76190476 0.82539683]
Average accuracy: 0.7873015873015872 

kernel poly
Cross-validation scores: [0.9047619  0.80952381 0.69047619 0.85714286 0.85714286]
Average accuracy: 0.8238095238095239
Cross-validation scores: [0.79365079 0.77777778 0.85714286 0.82539683 0.84126984]
Average accuracy: 0.819047619047619 

kernel rbf
Cross-validation scores: [0.88095238 0.78571429 0.71428571 0.76190476 0.88095238]
Average accuracy: 0.8047619047619048
Cross-validation scores: [0.82539683 0.76190476 0.85714286 0.76190476 0.87301587]
Average accuracy: 0.8158730158730159 



## --------FNN-----------------------------------------------------------------------------------------------------------

In [202]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

epochs = [10, 25, 50, 100, 500, 1000]
test_resultsNN = pd.DataFrame()
x = [0, 1, 2, 3]

for layers in x:
    for epoch in epochs:
        result = []
        # the model for real data:
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

        model = Sequential()
        for i in range(layers):
            if i == 0:
                model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
            else:
                model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
            model.add(Dropout(0.2))
        model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
        # output layer:
        model.add(Dense(3, activation='softmax'))

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_x, train_y, epochs=epoch, batch_size=13, validation_split=0.2, callbacks=[early_stopping])
        loss, accuracy = model.evaluate(test_x, test_y)
        print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
        result.append(accuracy)

        #for agum data:
        model = Sequential()
        for i in range(layers):
            if i == 0:
                model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
            else:
                model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
            model.add(Dropout(0.2))
        model.add(Dense(50, activation='relu', input_shape=(train_x.shape[1],)))
        # output layer:
        model.add(Dense(3, activation='softmax')) 

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(trainAgg_x, trainAgg_y, epochs=epoch, batch_size=13, validation_split=0.2, callbacks=[early_stopping])
        loss, accuracy = model.evaluate(test_x, test_y)
        print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
        result.append(accuracy)

        test_resultsNN["NN"+str(layers) +"_"+str(epoch)]= result

Epoch 1/10
24/24 [==============================] - 1s 8ms/step - loss: 1.0716 - accuracy: 0.4148 - val_loss: 1.0054 - val_accuracy: 0.4872
Epoch 2/10
24/24 [==============================] - 0s 3ms/step - loss: 0.9814 - accuracy: 0.5498 - val_loss: 0.9467 - val_accuracy: 0.6026
Epoch 3/10
24/24 [==============================] - 0s 3ms/step - loss: 0.9197 - accuracy: 0.6367 - val_loss: 0.8851 - val_accuracy: 0.7308
Epoch 4/10
24/24 [==============================] - 0s 3ms/step - loss: 0.8629 - accuracy: 0.7042 - val_loss: 0.8249 - val_accuracy: 0.7564
Epoch 5/10
24/24 [==============================] - 0s 3ms/step - loss: 0.8036 - accuracy: 0.7685 - val_loss: 0.7566 - val_accuracy: 0.7821
Epoch 6/10
24/24 [==============================] - 0s 3ms/step - loss: 0.7485 - accuracy: 0.7974 - val_loss: 0.6869 - val_accuracy: 0.7949
Epoch 7/10
24/24 [==============================] - 0s 3ms/step - loss: 0.6979 - accuracy: 0.7942 - val_loss: 0.6381 - val_accuracy: 0.7821
Epoch 8/10
24/24 [==

24/24 [==============================] - 0s 3ms/step - loss: 0.3447 - accuracy: 0.8553 - val_loss: 0.4174 - val_accuracy: 0.7821
Epoch 44/50
24/24 [==============================] - 0s 4ms/step - loss: 0.3432 - accuracy: 0.8521 - val_loss: 0.4136 - val_accuracy: 0.7949
Epoch 45/50
24/24 [==============================] - 0s 4ms/step - loss: 0.3426 - accuracy: 0.8617 - val_loss: 0.4139 - val_accuracy: 0.7821
Epoch 46/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3404 - accuracy: 0.8521 - val_loss: 0.4088 - val_accuracy: 0.7821
Epoch 47/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3373 - accuracy: 0.8585 - val_loss: 0.4119 - val_accuracy: 0.7692
Epoch 48/50
24/24 [==============================] - 0s 2ms/step - loss: 0.3312 - accuracy: 0.8585 - val_loss: 0.4107 - val_accuracy: 0.7692
Epoch 49/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3295 - accuracy: 0.8682 - val_loss: 0.4060 - val_accuracy: 0.7949
Epoch 50/50
13/13 [======

Epoch 57/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3384 - accuracy: 0.8553 - val_loss: 0.4091 - val_accuracy: 0.7949
Epoch 58/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3364 - accuracy: 0.8489 - val_loss: 0.4121 - val_accuracy: 0.7564
Epoch 59/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3353 - accuracy: 0.8650 - val_loss: 0.4080 - val_accuracy: 0.7692
Epoch 60/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3377 - accuracy: 0.8489 - val_loss: 0.4065 - val_accuracy: 0.7949
Epoch 61/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3315 - accuracy: 0.8585 - val_loss: 0.4087 - val_accuracy: 0.7564
Epoch 62/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3279 - accuracy: 0.8553 - val_loss: 0.4087 - val_accuracy: 0.7564
Epoch 63/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3277 - accuracy: 0.8650 - val_loss: 0.4064 - val_accuracy: 0.7692
Epoch 

Epoch 14/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4411 - accuracy: 0.8135 - val_loss: 0.5372 - val_accuracy: 0.7949
Epoch 15/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4388 - accuracy: 0.8039 - val_loss: 0.5307 - val_accuracy: 0.7949
Epoch 16/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4333 - accuracy: 0.8151 - val_loss: 0.5296 - val_accuracy: 0.8077
Epoch 17/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.8055 - val_loss: 0.5246 - val_accuracy: 0.8077
Epoch 18/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.8055 - val_loss: 0.5197 - val_accuracy: 0.7885
Epoch 19/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.8167 - val_loss: 0.5212 - val_accuracy: 0.8013
Epoch 20/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4206 - accuracy: 0.8167 - val_loss: 0.5255 - val_accuracy: 0.7949
Epoch 

Epoch 72/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3442 - accuracy: 0.8489 - val_loss: 0.4250 - val_accuracy: 0.8397
Epoch 73/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3406 - accuracy: 0.8521 - val_loss: 0.4143 - val_accuracy: 0.8269
Epoch 74/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3403 - accuracy: 0.8441 - val_loss: 0.4137 - val_accuracy: 0.8269
Epoch 75/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3420 - accuracy: 0.8473 - val_loss: 0.4179 - val_accuracy: 0.8205
Epoch 76/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3407 - accuracy: 0.8376 - val_loss: 0.4142 - val_accuracy: 0.8397
Epoch 77/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3363 - accuracy: 0.8489 - val_loss: 0.4075 - val_accuracy: 0.8205
Epoch 78/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8424 - val_loss: 0.4113 - val_accuracy: 0.8205
Epoch 

...

24/24 [==============================] - 0s 3ms/step - loss: 0.3573 - accuracy: 0.8392 - val_loss: 0.4707 - val_accuracy: 0.7564
Epoch 29/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3614 - accuracy: 0.8103 - val_loss: 0.4544 - val_accuracy: 0.8205
Epoch 30/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3320 - accuracy: 0.8553 - val_loss: 0.4655 - val_accuracy: 0.7564
Epoch 31/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3524 - accuracy: 0.8232 - val_loss: 0.4504 - val_accuracy: 0.8333
Epoch 31: early stopping
13/13 [==============================] - 0s 3ms/step - loss: 0.3674 - accuracy: 0.8210
Test Loss: 0.36738672852516174, Test Accuracy: 0.8209718465805054
Epoch 1/50
48/48 [==============================] - 1s 5ms/step - loss: 1.0426 - accuracy: 0.3907 - val_loss: 0.9697 - val_accuracy: 0.4231
Epoch 2/50
48/48 [==============================] - 0s 2ms/step - loss: 0.8637 - accuracy: 0.6013 - val_loss: 0.7555 - val_accurac

Epoch 2/100
24/24 [==============================] - 0s 3ms/step - loss: 0.9739 - accuracy: 0.4180 - val_loss: 0.8769 - val_accuracy: 0.5000
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 0.8499 - accuracy: 0.6495 - val_loss: 0.7798 - val_accuracy: 0.7564
Epoch 4/100
24/24 [==============================] - 0s 4ms/step - loss: 0.7415 - accuracy: 0.7524 - val_loss: 0.6505 - val_accuracy: 0.7821
Epoch 5/100
24/24 [==============================] - 0s 3ms/step - loss: 0.5921 - accuracy: 0.7621 - val_loss: 0.4827 - val_accuracy: 0.7692
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 0.5207 - accuracy: 0.8071 - val_loss: 0.4437 - val_accuracy: 0.7692
Epoch 7/100
24/24 [==============================] - 0s 3ms/step - loss: 0.4834 - accuracy: 0.7621 - val_loss: 0.4665 - val_accuracy: 0.7821
Epoch 8/100
24/24 [==============================] - 0s 3ms/step - loss: 0.4610 - accuracy: 0.7910 - val_loss: 0.4338 - val_accuracy: 0.7692
Epoch 9/100
2

Epoch 69: early stopping
13/13 [==============================] - 0s 1ms/step - loss: 0.2736 - accuracy: 0.8875
Test Loss: 0.2736451327800751, Test Accuracy: 0.8874680399894714
Epoch 1/500
24/24 [==============================] - 1s 9ms/step - loss: 1.0628 - accuracy: 0.3923 - val_loss: 0.9993 - val_accuracy: 0.4359
Epoch 2/500
24/24 [==============================] - 0s 3ms/step - loss: 0.9753 - accuracy: 0.4277 - val_loss: 0.8823 - val_accuracy: 0.6795
Epoch 3/500
24/24 [==============================] - 0s 3ms/step - loss: 0.8731 - accuracy: 0.6109 - val_loss: 0.7308 - val_accuracy: 0.8077
Epoch 4/500
24/24 [==============================] - 0s 4ms/step - loss: 0.7360 - accuracy: 0.7170 - val_loss: 0.5592 - val_accuracy: 0.7821
Epoch 5/500
24/24 [==============================] - 0s 3ms/step - loss: 0.6166 - accuracy: 0.7138 - val_loss: 0.5077 - val_accuracy: 0.7692
Epoch 6/500
24/24 [==============================] - 0s 3ms/step - loss: 0.6060 - accuracy: 0.7363 - val_loss: 0.4956 

Epoch 63/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2798 - accuracy: 0.8826 - val_loss: 0.3785 - val_accuracy: 0.8333
Epoch 64/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2946 - accuracy: 0.8746 - val_loss: 0.3748 - val_accuracy: 0.8462
Epoch 65/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2961 - accuracy: 0.8633 - val_loss: 0.3590 - val_accuracy: 0.8462
Epoch 66/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2868 - accuracy: 0.9019 - val_loss: 0.3921 - val_accuracy: 0.8397
Epoch 67/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2773 - accuracy: 0.8875 - val_loss: 0.3758 - val_accuracy: 0.8590
Epoch 68/500
48/48 [==============================] - 0s 3ms/step - loss: 0.2864 - accuracy: 0.8842 - val_loss: 0.3696 - val_accuracy: 0.8397
Epoch 69/500
48/48 [==============================] - 0s 2ms/step - loss: 0.2637 - accuracy: 0.8955 - val_loss: 0.3777 - val_accuracy: 0.8397
Epoch 

48/48 [==============================] - 0s 2ms/step - loss: 0.5131 - accuracy: 0.7524 - val_loss: 0.5832 - val_accuracy: 0.7756
Epoch 6/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4992 - accuracy: 0.7556 - val_loss: 0.5811 - val_accuracy: 0.7885
Epoch 7/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4836 - accuracy: 0.7846 - val_loss: 0.5841 - val_accuracy: 0.8013
Epoch 8/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4760 - accuracy: 0.7717 - val_loss: 0.5588 - val_accuracy: 0.7949
Epoch 9/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4608 - accuracy: 0.7958 - val_loss: 0.5708 - val_accuracy: 0.7244
Epoch 10/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4760 - accuracy: 0.7621 - val_loss: 0.5602 - val_accuracy: 0.7564
Epoch 11/1000
48/48 [==============================] - 0s 3ms/step - loss: 0.4677 - accuracy: 0.7797 - val_loss: 0.5551 - val_accuracy: 0.7821
Epoch 12/1000
48/

In [203]:
test_resultsNN

,NN0_10,NN0_25,NN0_50,NN0_100,NN0_500,NN0_1000,NN1_10,NN1_25,NN1_50,NN1_100,...,NN2_50,NN2_100,NN2_500,NN2_1000,NN3_10,NN3_25,NN3_50,NN3_100,NN3_500,NN3_1000
0,0.810742,0.851662,0.869565,0.869565,0.874680,0.872123,0.813299,0.856777,0.879795,0.900256,...,0.815857,0.867008,0.887468,0.836317,0.836317,0.867008,0.820972,0.864450,0.867008,0.872123
1,0.836317,0.841432,0.861893,0.869565,0.897698,0.895141,0.820972,0.861893,0.890026,0.887468,...,0.884910,0.887468,0.882353,0.913043,0.833760,0.843990,0.859335,0.887468,0.882353,0.879795


In [204]:
test_resultsNN.to_csv('early_stopping_eq_NN.csv', index=False)

## --------LSTM----------------------------------------------------------------------------------------------------------------

In [205]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

epochs = [10, 25, 50 , 100 , 500 ,1000]
test_resultsLSTM = pd.DataFrame()
x = [0,1,2,3] 
for layers in x:
    for epoch in epochs:
        result = []
        # the model for real data:
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
        model = Sequential()
        for i in range(layers):
            if i == 0:
                model.add(LSTM(50, input_shape=(train_x.shape[1], 1), return_sequences=True))
            else:
                model.add(LSTM(50, return_sequences=True))
            model.add(Dropout(0.2))
        model.add(LSTM(50))  # Add this to end the sequence
        # output layer:
        model.add(Dense(3, activation='softmax'))

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_x, train_y, epochs=epoch, batch_size=13, validation_split=0.2, callbacks=[early_stopping])
        loss, accuracy = model.evaluate(test_x, test_y)
        print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
        result.append(accuracy)

        #for agum data:
        model = Sequential()
        for i in range(layers):
            if i == 0:
                model.add(LSTM(50, input_shape=(train_x.shape[1], 1), return_sequences=True))
            else:
                model.add(LSTM(50, return_sequences=True))
            model.add(Dropout(0.2))
        model.add(LSTM(50))  # Add this to end the sequence
        # output layer:
        model.add(Dense(3, activation='softmax')) 

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(trainAgg_x, trainAgg_y, epochs=epoch, batch_size=13, validation_split=0.2, callbacks=[early_stopping])
        loss, accuracy = model.evaluate(test_x, test_y)
        print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
        result.append(accuracy)

        test_resultsLSTM["LSTM"+str(layers) +"_"+str(epoch)]= result

Epoch 1/10
24/24 [==============================] - 5s 25ms/step - loss: 1.0703 - accuracy: 0.3666 - val_loss: 1.0151 - val_accuracy: 0.3846
Epoch 2/10
24/24 [==============================] - 0s 8ms/step - loss: 0.9228 - accuracy: 0.4598 - val_loss: 0.7722 - val_accuracy: 0.7436
Epoch 3/10
24/24 [==============================] - 0s 8ms/step - loss: 0.7401 - accuracy: 0.6720 - val_loss: 0.5796 - val_accuracy: 0.7308
Epoch 4/10
24/24 [==============================] - 0s 7ms/step - loss: 0.6585 - accuracy: 0.6945 - val_loss: 0.4863 - val_accuracy: 0.7308
Epoch 5/10
24/24 [==============================] - 0s 7ms/step - loss: 0.6225 - accuracy: 0.7267 - val_loss: 0.4989 - val_accuracy: 0.7436
Epoch 6/10
24/24 [==============================] - 0s 8ms/step - loss: 0.6317 - accuracy: 0.7106 - val_loss: 0.4594 - val_accuracy: 0.7564
Epoch 7/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5971 - accuracy: 0.7299 - val_loss: 0.4871 - val_accuracy: 0.7436
Epoch 8/10
24/24 [=

Epoch 18/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5566 - accuracy: 0.7395 - val_loss: 0.6280 - val_accuracy: 0.7115
Epoch 19/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5517 - accuracy: 0.7444 - val_loss: 0.6305 - val_accuracy: 0.7244
Epoch 20/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5699 - accuracy: 0.7267 - val_loss: 0.6383 - val_accuracy: 0.7051
Epoch 21/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5602 - accuracy: 0.7315 - val_loss: 0.6256 - val_accuracy: 0.7244
Epoch 22/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5587 - accuracy: 0.7331 - val_loss: 0.6235 - val_accuracy: 0.7115
Epoch 23/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5572 - accuracy: 0.7347 - val_loss: 0.6296 - val_accuracy: 0.7372
Epoch 24/25
48/48 [==============================] - 0s 8ms/step - loss: 0.5478 - accuracy: 0.7315 - val_loss: 0.6433 - val_accuracy: 0.7308
Epoch 25/25
1

48/48 [==============================] - 0s 7ms/step - loss: 0.5599 - accuracy: 0.7460 - val_loss: 0.6434 - val_accuracy: 0.7115
Epoch 28: early stopping
13/13 [==============================] - 0s 4ms/step - loss: 0.5197 - accuracy: 0.7596
Test Loss: 0.519705593585968, Test Accuracy: 0.7595908045768738
Epoch 1/100
24/24 [==============================] - 2s 24ms/step - loss: 1.0859 - accuracy: 0.3601 - val_loss: 1.0731 - val_accuracy: 0.2308
Epoch 2/100
24/24 [==============================] - 0s 8ms/step - loss: 0.9452 - accuracy: 0.4469 - val_loss: 0.6608 - val_accuracy: 0.7308
Epoch 3/100
24/24 [==============================] - 0s 8ms/step - loss: 0.6957 - accuracy: 0.6238 - val_loss: 0.5004 - val_accuracy: 0.7308
Epoch 4/100
24/24 [==============================] - 0s 8ms/step - loss: 0.6707 - accuracy: 0.6527 - val_loss: 0.5186 - val_accuracy: 0.7308
Epoch 5/100
24/24 [==============================] - 0s 8ms/step - loss: 0.6218 - accuracy: 0.6881 - val_loss: 0.4554 - val_accura

Epoch 93/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4587 - accuracy: 0.7685 - val_loss: 0.5630 - val_accuracy: 0.7436
Epoch 94/100
48/48 [==============================] - 0s 7ms/step - loss: 0.4560 - accuracy: 0.7797 - val_loss: 0.5939 - val_accuracy: 0.7436
Epoch 95/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4941 - accuracy: 0.7637 - val_loss: 0.5698 - val_accuracy: 0.7436
Epoch 96/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4554 - accuracy: 0.7846 - val_loss: 0.5455 - val_accuracy: 0.7949
Epoch 97/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4537 - accuracy: 0.7878 - val_loss: 0.5716 - val_accuracy: 0.7564
Epoch 98/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4553 - accuracy: 0.7797 - val_loss: 0.5658 - val_accuracy: 0.7692
Epoch 99/100
48/48 [==============================] - 0s 8ms/step - loss: 0.4658 - accuracy: 0.7669 - val_loss: 0.5701 - val_accuracy: 0.7564
Epoch 

Epoch 84/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5142 - accuracy: 0.7701 - val_loss: 0.6184 - val_accuracy: 0.7436
Epoch 85/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5096 - accuracy: 0.7492 - val_loss: 0.6127 - val_accuracy: 0.7436
Epoch 86/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5170 - accuracy: 0.7588 - val_loss: 0.6026 - val_accuracy: 0.7500
Epoch 87/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5182 - accuracy: 0.7572 - val_loss: 0.6144 - val_accuracy: 0.7308
Epoch 88/500
48/48 [==============================] - 0s 8ms/step - loss: 0.5164 - accuracy: 0.7476 - val_loss: 0.6052 - val_accuracy: 0.7564
Epoch 89/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5090 - accuracy: 0.7605 - val_loss: 0.5977 - val_accuracy: 0.7372
Epoch 90/500
48/48 [==============================] - 0s 7ms/step - loss: 0.5073 - accuracy: 0.7556 - val_loss: 0.6015 - val_accuracy: 0.7500
Epoch 

...

24/24 [==============================] - 0s 20ms/step - loss: 0.5877 - accuracy: 0.7203 - val_loss: 0.4744 - val_accuracy: 0.7692
Epoch 9/1000
24/24 [==============================] - 1s 21ms/step - loss: 0.5859 - accuracy: 0.7106 - val_loss: 0.4919 - val_accuracy: 0.7692
Epoch 10/1000
24/24 [==============================] - 1s 21ms/step - loss: 0.5819 - accuracy: 0.7363 - val_loss: 0.4864 - val_accuracy: 0.7692
Epoch 11/1000
24/24 [==============================] - 0s 19ms/step - loss: 0.5671 - accuracy: 0.7363 - val_loss: 0.4789 - val_accuracy: 0.7821
Epoch 12/1000
24/24 [==============================] - 1s 22ms/step - loss: 0.5805 - accuracy: 0.7106 - val_loss: 0.4811 - val_accuracy: 0.7564
Epoch 13/1000
24/24 [==============================] - 1s 22ms/step - loss: 0.5845 - accuracy: 0.7203 - val_loss: 0.4736 - val_accuracy: 0.7692
Epoch 13: early stopping
13/13 [==============================] - 1s 7ms/step - loss: 0.5359 - accuracy: 0.7263
Test Loss: 0.53591388463974, Test Accur

24/24 [==============================] - 1s 25ms/step - loss: 0.5574 - accuracy: 0.7588 - val_loss: 0.4728 - val_accuracy: 0.7436
Epoch 32/50
24/24 [==============================] - 1s 28ms/step - loss: 0.5770 - accuracy: 0.7299 - val_loss: 0.4846 - val_accuracy: 0.7564
Epoch 33/50
24/24 [==============================] - 1s 27ms/step - loss: 0.5660 - accuracy: 0.7363 - val_loss: 0.5001 - val_accuracy: 0.7564
Epoch 34/50
24/24 [==============================] - 1s 29ms/step - loss: 0.5628 - accuracy: 0.7267 - val_loss: 0.4867 - val_accuracy: 0.7692
Epoch 35/50
24/24 [==============================] - 1s 27ms/step - loss: 0.5860 - accuracy: 0.7331 - val_loss: 0.4918 - val_accuracy: 0.7692
Epoch 36/50
24/24 [==============================] - 1s 26ms/step - loss: 0.5711 - accuracy: 0.7428 - val_loss: 0.5057 - val_accuracy: 0.7564
Epoch 37/50
24/24 [==============================] - 1s 27ms/step - loss: 0.5655 - accuracy: 0.7492 - val_loss: 0.4890 - val_accuracy: 0.7692
Epoch 38/50
24/24 

48/48 [==============================] - 1s 26ms/step - loss: 0.5818 - accuracy: 0.7283 - val_loss: 0.6622 - val_accuracy: 0.7115
Epoch 9/100
48/48 [==============================] - 1s 26ms/step - loss: 0.5699 - accuracy: 0.7235 - val_loss: 0.7261 - val_accuracy: 0.6731
Epoch 10/100
48/48 [==============================] - 1s 29ms/step - loss: 0.5745 - accuracy: 0.7106 - val_loss: 0.6430 - val_accuracy: 0.7179
Epoch 11/100
48/48 [==============================] - 1s 28ms/step - loss: 0.5965 - accuracy: 0.7235 - val_loss: 0.6470 - val_accuracy: 0.7244
Epoch 12/100
48/48 [==============================] - 1s 29ms/step - loss: 0.5772 - accuracy: 0.7219 - val_loss: 0.6657 - val_accuracy: 0.7500
Epoch 13/100
48/48 [==============================] - 1s 29ms/step - loss: 0.5642 - accuracy: 0.7412 - val_loss: 0.6452 - val_accuracy: 0.7372
Epoch 14/100
48/48 [==============================] - 1s 28ms/step - loss: 0.5591 - accuracy: 0.7267 - val_loss: 0.6577 - val_accuracy: 0.7244
Epoch 15/100


24/24 [==============================] - 1s 26ms/step - loss: 0.6017 - accuracy: 0.7010 - val_loss: 0.4886 - val_accuracy: 0.7692
Epoch 22/500
24/24 [==============================] - 1s 25ms/step - loss: 0.5890 - accuracy: 0.7010 - val_loss: 0.5038 - val_accuracy: 0.7564
Epoch 22: early stopping
13/13 [==============================] - 2s 12ms/step - loss: 0.5358 - accuracy: 0.7161
Test Loss: 0.5357652306556702, Test Accuracy: 0.7161125540733337
Epoch 1/500
48/48 [==============================] - 8s 50ms/step - loss: 0.8012 - accuracy: 0.5498 - val_loss: 0.6751 - val_accuracy: 0.7308
Epoch 2/500
48/48 [==============================] - 1s 25ms/step - loss: 0.5882 - accuracy: 0.7315 - val_loss: 0.6452 - val_accuracy: 0.7051
Epoch 3/500
48/48 [==============================] - 1s 24ms/step - loss: 0.5875 - accuracy: 0.7283 - val_loss: 0.6426 - val_accuracy: 0.6987
Epoch 4/500
48/48 [==============================] - 1s 25ms/step - loss: 0.5793 - accuracy: 0.7219 - val_loss: 0.6477 - va

Epoch 14/1000
24/24 [==============================] - 1s 33ms/step - loss: 0.5685 - accuracy: 0.7235 - val_loss: 0.5137 - val_accuracy: 0.7564
Epoch 15/1000
24/24 [==============================] - 1s 30ms/step - loss: 0.5889 - accuracy: 0.7138 - val_loss: 0.5020 - val_accuracy: 0.7564
Epoch 16/1000
24/24 [==============================] - 1s 30ms/step - loss: 0.6282 - accuracy: 0.6945 - val_loss: 0.5208 - val_accuracy: 0.7564
Epoch 16: early stopping
13/13 [==============================] - 2s 11ms/step - loss: 0.5957 - accuracy: 0.6752
Test Loss: 0.5956622362136841, Test Accuracy: 0.6751918196678162
Epoch 1/1000
48/48 [==============================] - 11s 66ms/step - loss: 0.8687 - accuracy: 0.5579 - val_loss: 0.6781 - val_accuracy: 0.6731
Epoch 2/1000
48/48 [==============================] - 1s 31ms/step - loss: 0.6106 - accuracy: 0.7106 - val_loss: 0.6396 - val_accuracy: 0.7179
Epoch 3/1000
48/48 [==============================] - 1s 30ms/step - loss: 0.5890 - accuracy: 0.7331 - 

In [206]:
test_resultsLSTM

,LSTM0_10,LSTM0_25,LSTM0_50,LSTM0_100,LSTM0_500,LSTM0_1000,LSTM1_10,LSTM1_25,LSTM1_50,LSTM1_100,...,LSTM2_50,LSTM2_100,LSTM2_500,LSTM2_1000,LSTM3_10,LSTM3_25,LSTM3_50,LSTM3_100,LSTM3_500,LSTM3_1000
0,0.741688,0.734015,0.744246,0.749361,0.751918,0.721228,0.723785,0.716113,0.762148,0.703325,...,0.749361,0.723785,0.744246,0.726343,0.739130,0.728900,0.713555,0.739130,0.716113,0.675192
1,0.718670,0.728900,0.759591,0.800511,0.797954,0.751918,0.749361,0.772379,0.728900,0.820972,...,0.746803,0.838875,0.849105,0.759591,0.723785,0.764706,0.762148,0.851662,0.759591,0.759591


In [207]:
test_resultsLSTM.to_csv('early_stopping_eq_LSTM.csv', index=False)